In [79]:
from tkinter import Tk, filedialog
import pandas as pd
import ast

In [80]:
# Create Tk root
root = Tk()
# Hide the main window
root.withdraw()
root.call('wm', 'attributes', '.', '-topmost', True)

# Ask user to select CSV files
csv_paths = filedialog.askopenfilename(multiple=True, title='Select CSV files', filetypes=[('CSV files', '*.csv')])

In [81]:
# Check if files were selected
if csv_paths:
    # Initialize an empty DataFrame to store all the data
    all_data = pd.DataFrame()

    for csv_path in csv_paths:
        # Read the selected CSV file
        data = pd.read_csv(csv_path)

        # Append the data to the all_data DataFrame
        all_data = all_data.append(data, ignore_index=True)

    # Get unique values for Genotype, Session Number, Stim Behavior, Fiber Connection, and Sham
    unique_name= all_data['Mouse'].unique()
    unique_genotypes = all_data['Genotype'].unique()
    unique_sessions = all_data['Session Number'].unique()
    unique_selected_behaviors = all_data['Selected Behavior'].unique()
    unique_connections = all_data['Fiber Connection'].unique()

    # Print the unique values for reference
    #print("Unique Name:",unique_name)
    #print("Unique Genotypes:", unique_genotypes)
    #print("Unique Sessions:", unique_sessions)
    #print("Unique Selected Behaviors:", unique_selected_behaviors)
    #print("Unique Connections:", unique_connections)

    # Prompt the user to enter criteria
    selected_name= input(f"Enter Mouse Name(or press Enter to skip): ").lower()
    selected_genotype = input(f"Enter Genotype ({', '.join(unique_genotypes)} or press Enter to skip): ").lower()
    selected_session = input(f"Enter Session Number ({', '.join(map(str, unique_sessions))} or press Enter to skip): ").lower()
    selected_behavior = input(f"Enter Stim Behavior ({', '.join(unique_selected_behaviors)} or press Enter to skip): ").lower()
    selected_connection = input(f"Enter Fiber Connection ({', '.join(unique_connections)} or press Enter to skip): ").lower()
    selected_sham_input = input("Enter Sham (y for Yes, n for No) or press Enter to skip: ").lower()

    # Convert the user input to boolean
    selected_sham = True if selected_sham_input == 'y' else False

  # Filter the data based on the selected criteria
if selected_name == "" and selected_genotype == "" and selected_session == "" and selected_behavior == "" and selected_connection == "" and selected_sham_input == "":
    # No specific filters provided, so all animals meet the requirements
    selected_data = all_data
else:
    selected_data = all_data[
        (selected_name == "" or all_data['Mouse'].str.lower() == selected_name) &
        (selected_genotype == "" or all_data['Genotype'].str.lower() == selected_genotype) &
        (selected_session == "" or all_data['Session Number'] == int(selected_session)) &
        (selected_behavior == "" or all_data['Selected Behavior'].str.lower() == selected_behavior) &
        (selected_connection == "" or all_data['Fiber Connection'].str.lower() == selected_connection) &
        (selected_sham_input == "" or all_data['Sham'] == selected_sham)
    ]

# Display the animals that meet the criteria
if not selected_data.empty:
    print("Animals that meet the criteria:")
    print(len(selected_data))
else:
    print("No animals meet the criteria.")

C:\Users\Yttri-Lab\AppData\Local\Temp\ipykernel_15672\3665171427.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data, ignore_index=True)


Enter Mouse Name(or press Enter to skip): 
Enter Genotype (D1, D2 or press Enter to skip): 
Enter Session Number (1, 2 or press Enter to skip): 
Enter Stim Behavior (Locomotion , Right Turn , Face Groom  or press Enter to skip): 
Enter Fiber Connection (Bilateral, Contralateral, Ipsilateral or press Enter to skip): 
Enter Sham (y for Yes, n for No) or press Enter to skip: 
Animals that meet the criteria:
15


In [85]:
#Filter columns with 'Array' in their names
array_columns = selected_data.filter(like='Array')

# Function to convert the string representation of a list of tuples to an actual list of tuples
def convert_string_to_list_of_tuples(s):
    try:
        return ast.literal_eval(s)
    except (SyntaxError, ValueError):
        return []

# Convert 'Array' columns to lists of tuples
for column_name in array_columns.columns:
    selected_data[column_name] = selected_data[column_name].apply(convert_string_to_list_of_tuples)

In [106]:
# Filter columns with 'Array' in their names
array_columns = selected_data.filter(like='Array')

# Iterate through rows of the filtered columns
for animal_id, row in array_columns.iterrows():
    mouse_name = selected_data.loc[animal_id, 'Mouse']  
    genotype = selected_data.loc[animal_id, 'Genotype']
    date = selected_data.loc[animal_id, 'Date']
    behavior = selected_data.loc[animal_id, 'Selected Behavior']
    #print(f"Mouse Name: {mouse_name}, Genotype: {genotype}, Date: {date}, Selected Behavior: {behavior}")
    #print("Extracted Columns:")
    #print(row)
    #print("-----------------------------------------")

# Get unique behaviors in the dataset
unique_behaviors = selected_data['Selected Behavior'].unique()

if len(unique_behaviors) > 1:
    print("Different behaviors found in the dataset:")
    for behavior in unique_behaviors:
        print(behavior)

    # Prompt user for desired behavior, or press Enter to consider all behaviors
    desired_behavior = input("Enter the desired behavior (or press Enter to consider all behaviors): ")

    # Convert the desired behavior to lowercase and strip leading/trailing whitespace
    desired_behavior = desired_behavior.strip().lower()

    # Filter through the selected behavior and store matching rows
    matching_rows = []
    for animal_id, row in array_columns.iterrows():
        behavior = selected_data.loc[animal_id, 'Selected Behavior'].strip().lower()
        if not desired_behavior or behavior == desired_behavior:
            matching_rows.append(row)

    # If no matches were found
    if not matching_rows:
        print('No animals match')
else:
    print("There's only one behavior in the dataset. No need to filter.")

    

Different behaviors found in the dataset:
Locomotion 
Right Turn 
Face Groom 
Enter the desired behavior (or press Enter to consider all behaviors): 


In [101]:
if matching_rows:
    # Get a list of available array columns from the first matching row
    first_matching_row = matching_rows[0]
    array_column_names = first_matching_row.index.tolist()

    # Display available array column options to the user
    print("Available array columns:")
    for index, column in enumerate(array_column_names):
        print(f"{index + 1}. {column}")

    # Prompt user to select an array column
    selected_array_index = int(input("Enter the number of the array column you'd like to filter through: ")) - 1
    selected_array_column = array_column_names[selected_array_index]

    # Filter through the selected array column and display results
    print(f"Results for selected array column '{selected_array_column}':")
    
    for row in matching_rows:
        mouse_name = selected_data.loc[row.name, 'Mouse']  
        genotype = selected_data.loc[row.name, 'Genotype']
        date = selected_data.loc[row.name, 'Date']
        behavior = selected_data.loc[row.name, 'Selected Behavior']

        # Extract the selected array column data for the current row
        selected_array_data = row[selected_array_column]
        print(selected_array_data)
        print(f"Mouse Name: {mouse_name}, Genotype: {genotype}, Date: {date}, Selected Behavior: {behavior}")
        print(f"Extracted Columns from {selected_array_column}:")
        for behavior_tuple in selected_array_data:
            behavior_info = {
                'Behavior': behavior_tuple[0],
                'Duration': behavior_tuple[1],
                'Bout': behavior_tuple[2],
                'Time': behavior_tuple[3]
            }
            print("Behavior Info:", behavior_info)
            print("-----------------------------------------")


Available array columns:
1. Right Turn Array
2. Locomotion Array
3. Face Groom Array
Enter the number of the array column you'd like to filter through: 2
Results for selected array column 'Locomotion Array':
[]
Mouse Name: AD10, Genotype: D1, Date: 62623, Selected Behavior: Locomotion 
Extracted Columns from Locomotion Array:
[]
Mouse Name: AD10, Genotype: D1, Date: 72723, Selected Behavior: Locomotion 
Extracted Columns from Locomotion Array:
[]
Mouse Name: AD10, Genotype: D1, Date: 71723, Selected Behavior: Right Turn 
Extracted Columns from Locomotion Array:
[]
Mouse Name: AD10, Genotype: D1, Date: 71123, Selected Behavior: Right Turn 
Extracted Columns from Locomotion Array:
[]
Mouse Name: AD6, Genotype: D2, Date: 51223, Selected Behavior: Face Groom 
Extracted Columns from Locomotion Array:
[]
Mouse Name: AD6, Genotype: D2, Date: 51123, Selected Behavior: Locomotion 
Extracted Columns from Locomotion Array:
[]
Mouse Name: AD6, Genotype: D2, Date: 72723, Selected Behavior: Locomoti

In [95]:
# Display available tuple elements to the user
print("Available tuple elements:")
for index, element in enumerate(['Behavior', 'Duration', 'Bout', 'Time']):
    print(f"{index + 1}. {element}")

# Prompt user to select elements to filter
selected_element_indices = input("Enter the numbers of the elements you'd like to filter (comma-separated, e.g., 1,2,3): ")
selected_element_indices = [int(idx) - 1 for idx in selected_element_indices.split(',')]

# Create a dictionary to store desired values for selected elements
desired_values = {}
for index in selected_element_indices:
    element = ['Behavior', 'Duration', 'Bout', 'Time'][index]
    desired_value = input(f"Enter the desired value for '{element}' (press Enter for all values): ")
    desired_values[index] = desired_value

# Initialize a dictionary to store results
results = {}

for animal_id in matching_rows:
    mouse_name = selected_data.loc[animal_id, 'Mouse']  
    genotype = selected_data.loc[animal_id, 'Genotype']
    date = selected_data.loc[animal_id, 'Date']
    behavior = selected_data.loc[animal_id, 'Selected Behavior']
    
    selected_array_data = array_columns.loc[animal_id, selected_array_column]
    
    for behavior_tuple in selected_array_data:
        matches = True
        for index, desired_value in desired_values.items():
            element_value = behavior_tuple[index]
            if desired_value and str(element_value) != desired_value:
                matches = False
                break
            
        if matches:
            if animal_id in results:
                results[animal_id].append(behavior_tuple)
            else:
                results[animal_id] = [behavior_tuple]

# Print the results
for animal_id, behaviors in results.items():
    print(f"Animal ID: {animal_id}")
    print(f"Mouse Name: {mouse_name}, Genotype: {genotype}, Date: {date}, Selected Behavior: {behavior}")
    print("Matched Elements and Values:")
    for index, desired_value in desired_values.items():
        if desired_value:
            element_name = ['Behavior', 'Duration', 'Bout', 'Time'][index]
            print(f"{element_name}: {desired_value}")
    print(f"Total occurrences matching all selected elements: {len(behaviors)}")
    for behavior_tuple in behaviors:
        print(f"Behavior: {behavior_tuple[0]}, Duration: {behavior_tuple[1]}, Bout: {behavior_tuple[2]}, Time: {behavior_tuple[3]}")
    print("-----------------------------------------")

Available array columns:
1. Right Turn Array
2. Locomotion Array
3. Face Groom Array
Enter the number of the array column you'd like to filter through: 1


TypeError: unhashable type: 'list'

In [45]:

# Create an empty list to store the binned stim counts for each animal and behavior
binned_stim_counts = []

# Iterate through the rows of duration_and_time_columns
for animal_id, behavior_data_list in duration_and_time_columns.iterrows():
    mouse_name = selected_data.loc[animal_id, 'Mouse']
    genotype = selected_data.loc[animal_id, 'Genotype']
    print(f"Calculating stim counts for Mouse Name: {mouse_name}, Genotype: {genotype}")

    stim_counts_per_behavior = {}  # Dictionary to store stim counts for each behavior

    # Iterate through each behavior data list in the row
    for behavior_name, behavior_data_str in behavior_data_list.items():
        if behavior_name not in ['Mouse', 'Genotype']:
            behavior_data = eval(behavior_data_str)  # Convert the string back to a list of tuples
            end_times = [duration_and_time[2] for duration_and_time in behavior_data]

            # Set up time interval parameters
            time_interval_start = 0
            time_interval_end = 3000  # Set the initial interval duration (adjust as needed)

            stim_count = 0  # Initialize stim count for this interval

            # Iterate through end times and bin stims
            for end_time in end_times:
                if end_time <= time_interval_end:
                    stim_count += 1
                else:
                    if behavior_name not in stim_counts_per_behavior:
                        stim_counts_per_behavior[behavior_name] = []
                    stim_counts_per_behavior[behavior_name].append(stim_count)
                    stim_count = 1  # Start a new stim count for the next interval
                    while end_time > time_interval_end:
                        time_interval_start = time_interval_end
                        time_interval_end += 3000  # Increase the interval duration (adjust as needed)
                    if end_time <= time_interval_end:
                        stim_count += 1

            if behavior_name not in stim_counts_per_behavior:
                stim_counts_per_behavior[behavior_name] = []
            stim_counts_per_behavior[behavior_name].append(stim_count)  # Append the last stim count

    # Append the stim counts for each behavior for this animal to the list
    binned_stim_counts.append(stim_counts_per_behavior)

    print("Stim Counts Over Time Intervals for Each Behavior:")
    print(stim_counts_per_behavior)
    print("-----------------------------------------")

# Calculate the average stim counts for each time period (interval)
average_interval_stim_counts = {}
for behavior_stim_counts in binned_stim_counts:
    for behavior, interval_counts in behavior_stim_counts.items():
        if behavior not in average_interval_stim_counts:
            average_interval_stim_counts[behavior] = [0] * len(interval_counts)
        average_interval_stim_counts[behavior] = [avg + count for avg, count in zip(average_interval_stim_counts[behavior], interval_counts)]

# Calculate the final average for each time period
for behavior, interval_counts in average_interval_stim_counts.items():
    num_animals = len(binned_stim_counts)
    average_interval_stim_counts[behavior] = [count / num_animals for count in interval_counts]

# Print the average stim counts for each time period (interval)
print("Average Stim Counts for Each Time Period:")
print(average_interval_stim_counts)

Calculating stim counts for Mouse Name: AD6, Genotype: D2
Stim Counts Over Time Intervals for Each Behavior:
{'Right Turn Duration and Time': [25, 39, 21, 35, 20, 28, 33, 27, 8, 7, 3, 5], 'Locomotion Duration and Time': [85, 92, 63, 74, 61, 46, 56, 40, 21, 9, 13, 7], 'Face Groom Duration and Time': [12, 6, 14, 7, 4, 19, 9, 15, 36, 15, 53]}
-----------------------------------------
Calculating stim counts for Mouse Name: AD6, Genotype: D2
Stim Counts Over Time Intervals for Each Behavior:
{'Right Turn Duration and Time': [29, 13, 14, 13, 10, 17, 4, 26, 11, 16, 24, 21, 22], 'Locomotion Duration and Time': [56, 19, 30, 24, 21, 19, 6, 28, 16, 18, 38, 41, 48], 'Face Groom Duration and Time': [1, 8, 3, 14, 3, 3, 3, 2, 4]}
-----------------------------------------
Calculating stim counts for Mouse Name: AD6, Genotype: D2
Stim Counts Over Time Intervals for Each Behavior:
{'Right Turn Duration and Time': [19, 18, 23, 18, 10, 16, 19, 13, 22, 21, 15, 27, 26], 'Locomotion Duration and Time': [40,